In [73]:
import os
import torch
import torch.nn as nn
from model import StyledGenerator, Discriminator, TextProcess
from dataset import MultiResolutionDataset
from train import sample_data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from pytorch_pretrained_bert import BertTokenizer
from PIL import Image
from torchvision.transforms import functional as trans_fn
from torchvision import transforms
from functools import partial

In [83]:
x = torch.rand(16,128)

In [92]:
x.view(16,-1,1,1).repeat(1,1,4,4).shape

torch.Size([16, 128, 4, 4])

In [93]:
16384/256

64.0

In [31]:
import lmdb
from io import BytesIO

In [32]:
env = lmdb.open(
            './data/birds_lmdb/',
            max_readers=32,
            readonly=True,
            lock=False,
            readahead=False,
            meminit=False,
        )

In [54]:
with env.begin(write=False) as txn:
    img_key = f'4-{str(0).zfill(5)}'.encode('utf-8')
    txt_key = f'txt-1-{str(0).zfill(5)}'.encode('utf-8')
    txt_bytes = txn.get(txt_key)
    txt = txt_bytes.decode('utf-8')
    img_bytes = txn.get(img_key)
    buffer = BytesIO(img_bytes)
    img = Image.open(buffer)

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [187]:
text_process = TextProcess()

In [14]:
# data loader test
transform = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
        ]
    )
dataset = MultiResolutionDataset('./birds/', transform)
loader = sample_data(dataset, 16, 8)
data_iter = iter(loader)

In [55]:
def sample_data(dataset, batch_size, image_size=4):
    dataset.resolution = image_size
    loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=4)

    return loader